# Multiclass SVM 구현

- 제출자: 20기 황태연
- 제출 일자: 2023.09.12. (화)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [3]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [4]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [5]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

# 0. One-hot Encoding

- 훈련 데이터는 위에서 확인을 했으니 정답 데이터를 확인해봅시다.

In [7]:
y_train.unique()

array(['virginica', 'versicolor', 'setosa'], dtype=object)

- 정답 데이터에는 'virginica', 'versicolor', 'setosa' 총 3개의 데이터가 있음을 확인했습니다.
- 이제 정답 데이터는 **one-hot encoding** 해줌으로써 모델이 학습할 수 있도록 전처리를 하겠습니다.

In [26]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

y_test

,setosa,versicolor,virginica
96,0,1,0
73,0,1,0
134,0,0,1
41,1,0,0
70,0,1,0
116,0,0,1
19,1,0,0
138,0,0,1
33,1,0,0
89,0,1,0


# 1. SVC(Support Vector Classification)

- sklearn.svm.SVC 모델은 Multiclass Classification까지 수행할 수 있는 분류용 SVM(Support Vector Machine)입니다.
- 그러나 이번 과제에서는 **SVC가 두 개의 class만을 분류할 수 있다**는 가정을 하고 활용해야 합니다.

- SVC를 사용하는 방법을 간단히 익힌 뒤에 다음 과정으로 넘어가겠습니다.

In [25]:
ex_SVM2 = SVC() # SVM 모델 정의
ex_SVM2.fit(X_train, y_train.iloc[:, 0]) # SVM 모델 훈련
ex_SVM2.predict(X_test) # SVM 모델 예측 (0: setosa가 아닌 것으로 예측, 1: setosa로 예측)

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0], dtype=uint8)

# 2. Multiclass SVM

- Multiclass SVM을 만드는 방법에는 **One vs One** 또는 **One vs Rest**가 있습니다.

## One vs Rest

- One vs Rest으로 Multiclass SVM을 만들어봅시다. 이때 정답 데이터는 One-hot encoding을 가정합니다.

In [103]:
class MulticlassSVM():
    def __init__(self):
        self.num_class = 0 # 정답 레이블의 개수 파악
        self.SVMs = []
        
    def fit(self, X_train, y_train):
        self.num_class = len(y_train.iloc[0, :])
        for i in range(self.num_class):
            model = SVC()
            model.fit(X_train, y_train.iloc[:, i])
            self.SVMs.append(model)
            
    def predict(self, X_test):
        preds = np.empty((len(X_test), 0))
        for i in range(self.num_class):
            pred = self.SVMs[i].predict(X_test)
            preds = np.hstack((preds, np.array([pred]).T))
        """
        preds 예시
        array([[1., 0., 0.],
               [0., 1., 1.],
               [0., 1., 0.],
               [0., 0., 0.]])
        """
        
        # 중복으로 classification한 것은 그 중 랜덤하게 하나를 선정
        for i, pred in enumerate(preds):
            if np.sum(pred) != 1:
                if np.sum(pred) > 1:
                    indices = np.where(pred == 1)[0]
                else:
                    indices = np.where(pred == 0)[0]
                index = np.random.choice(indices)
                preds[i] = np.zeros(self.num_class)
                preds[i][index] = 1
        """
        preds 예시
        array([[1., 0., 0.],
               [0., 0., 1.], < 둘 중 하나로 택함.
               [0., 1., 0.],
               [1., 0., 0.]]) < 셋 중 하나로 택함.
        """
        return preds

In [139]:
model = MulticlassSVM()
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy_score(y_test, pred)

0.9666666666666667

- pred에 랜덤성이 부여되었기 때문에 정확도는 매번 달라집니다. 대부분 0.933, 0.966, 1.0 중 하나의 값을 갖습니다.